# Download base model

In [1]:
from gpt_model import GPTModel

BASE_CONFIG = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "drop_rate": 0.0,        # Dropout rate
    "qkv_bias": True         # Query-key-value bias
}

model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

CHOOSE_MODEL = "gpt2-medium (355M)"

BASE_CONFIG.update(model_configs[CHOOSE_MODEL])

model_size = CHOOSE_MODEL.split(" ")[-1].lstrip("(").rstrip(")")
model = GPTModel(BASE_CONFIG)

In [2]:
from gpt2_base_download import download_and_load_gpt2
import torch 
from gpt_model import load_weights_into_gpt, text_to_token_ids, token_ids_to_text, generate
import tiktoken


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Loading GPT2 {model_size} model on {device}...")
settings, params = download_and_load_gpt2(model_size=model_size, models_dir="gpt2")
base_cfg = {
    "vocab_size": settings["n_vocab"],
    "context_length": settings["n_ctx"],
    "emb_dim": settings["n_embd"],
    "n_layers": settings["n_layer"],
    "n_heads": settings["n_head"],
    "drop_rate": settings.get("resid_pdrop", 0.0),
    "qkv_bias": True
}
print(base_cfg)

Loading GPT2 355M model on cuda...
File already exists and is up-to-date: gpt2\355M\checkpoint
File already exists and is up-to-date: gpt2\355M\encoder.json
File already exists and is up-to-date: gpt2\355M\hparams.json


model.ckpt.data-00000-of-00001: 100%|██████████| 1.42G/1.42G [36:30<00:00, 648kiB/s]
model.ckpt.index: 100%|██████████| 10.4k/10.4k [00:00<00:00, 2.60MiB/s]
model.ckpt.meta: 100%|██████████| 927k/927k [00:01<00:00, 593kiB/s]
vocab.bpe: 100%|██████████| 456k/456k [00:01<00:00, 439kiB/s]


{'vocab_size': 50257, 'context_length': 1024, 'emb_dim': 1024, 'n_layers': 24, 'n_heads': 16, 'drop_rate': 0.0, 'qkv_bias': True}


# Prompt base model

In [3]:
import os
import torch
import tiktoken

# Import from local files in this folder
from gpt_model import (
    GPTModel,
    generate,
    load_weights_into_gpt,
    text_to_token_ids,
    token_ids_to_text
)
from gpt2_base_download import download_and_load_gpt2

# Configuration
CHOOSE_MODEL = "gpt2-medium (355M)"  # Or choose "gpt2-small (124M)", "gpt2-large (774M)", "gpt2-xl (1558M)"
MODELS_DIR = r"C:\Users\YT40432\Desktop\lp\research\lucaspecina\ai-basics\models\gpt2" # Adjust this path if your models are stored elsewhere
PROMPT = "Instruction: Convert the active sentence to passive: 'The chef cooks the meal every day."
MAX_NEW_TOKENS = 50
TEMPERATURE = 0.7
TOP_K = 50

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(50*"-")

# Define model config based on chosen model
BASE_CONFIG = {
    "vocab_size": 50257,
    "context_length": 1024,
    "drop_rate": 0.0,
    "qkv_bias": True
}
model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}
if CHOOSE_MODEL not in model_configs:
    raise ValueError(f"Invalid model size. Choose from {list(model_configs.keys())}")

BASE_CONFIG.update(model_configs[CHOOSE_MODEL])

# Download and load model weights
print(f"Loading model: {CHOOSE_MODEL}")
model_size = CHOOSE_MODEL.split(" ")[-1].lstrip("(").rstrip(")")
settings, params = download_and_load_gpt2(model_size=model_size, models_dir=MODELS_DIR)

# Instantiate the model
model = GPTModel(BASE_CONFIG)
load_weights_into_gpt(model, params)
model.eval()
model.to(device)
print("Model loaded successfully.")
print(50*"-")

# Initialize tokenizer
tokenizer = tiktoken.get_encoding("gpt2")

# Tokenize the prompt
encoded_prompt = text_to_token_ids(PROMPT, tokenizer).to(device)

# Generate text
print(f"Prompt: >>>\n{PROMPT}\n<<<")
print(f"\nGenerating {MAX_NEW_TOKENS} tokens (temperature={TEMPERATURE}, top_k={TOP_K})...")
torch.manual_seed(123)  # For reproducibility
token_ids = generate(
    model=model,
    idx=encoded_prompt,
    max_new_tokens=MAX_NEW_TOKENS,
    context_size=BASE_CONFIG["context_length"],
    temperature=TEMPERATURE,
    top_k=TOP_K,
    eos_id=tokenizer.encode("<|endoftext|>")[0]
)

# Decode and print the generated text
decoded_text = token_ids_to_text(token_ids[0], tokenizer)
print(f"\nGenerated text: >>>\n{decoded_text}\n<<<")
print(50*"-")


Using device: cuda
--------------------------------------------------
Loading model: gpt2-medium (355M)
File already exists and is up-to-date: C:\Users\YT40432\Desktop\lp\research\lucaspecina\ai-basics\models\gpt2\355M\checkpoint
File already exists and is up-to-date: C:\Users\YT40432\Desktop\lp\research\lucaspecina\ai-basics\models\gpt2\355M\encoder.json
File already exists and is up-to-date: C:\Users\YT40432\Desktop\lp\research\lucaspecina\ai-basics\models\gpt2\355M\hparams.json
File already exists and is up-to-date: C:\Users\YT40432\Desktop\lp\research\lucaspecina\ai-basics\models\gpt2\355M\model.ckpt.data-00000-of-00001
File already exists and is up-to-date: C:\Users\YT40432\Desktop\lp\research\lucaspecina\ai-basics\models\gpt2\355M\model.ckpt.index
File already exists and is up-to-date: C:\Users\YT40432\Desktop\lp\research\lucaspecina\ai-basics\models\gpt2\355M\model.ckpt.meta
File already exists and is up-to-date: C:\Users\YT40432\Desktop\lp\research\lucaspecina\ai-basics\models\

ValueError: Encountered text corresponding to disallowed special token '<|endoftext|>'.
If you want this text to be encoded as a special token, pass it to `allowed_special`, e.g. `allowed_special={'<|endoftext|>', ...}`.
If you want this text to be encoded as normal text, disable the check for this token by passing `disallowed_special=(enc.special_tokens_set - {'<|endoftext|>'})`.
To disable this check for all special tokens, pass `disallowed_special=()`.


# Load And Use Finetuned Model

This notebook contains minimal code to load the finetuned model that was instruction finetuned and saved in chapter 7 via [ch07.ipynb](ch07.ipynb).

In [1]:
from importlib.metadata import version

pkgs = [
    "tiktoken",    # Tokenizer
    "torch",       # Deep learning library
]
for p in pkgs:
    print(f"{p} version: {version(p)}")

tiktoken version: 0.6.0
torch version: 2.6.0+cu124


In [2]:
from pathlib import Path

finetuned_model_path = Path("gpt2-medium355M-sft.pth")
if not finetuned_model_path.exists():
    print(
        f"Could not find '{finetuned_model_path}'.\n"
        "Run the `ch07.ipynb` notebook to finetune and save the finetuned model."
    )

Could not find 'gpt2-medium355M-sft.pth'.
Run the `ch07.ipynb` notebook to finetune and save the finetuned model.


In [3]:
from previous_chapters import GPTModel


BASE_CONFIG = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "drop_rate": 0.0,        # Dropout rate
    "qkv_bias": True         # Query-key-value bias
}

model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

CHOOSE_MODEL = "gpt2-medium (355M)"

BASE_CONFIG.update(model_configs[CHOOSE_MODEL])

model_size = CHOOSE_MODEL.split(" ")[-1].lstrip("(").rstrip(")")
model = GPTModel(BASE_CONFIG)

In [4]:
import torch

model.load_state_dict(torch.load(
    "gpt2-medium355M-sft.pth",
    map_location=torch.device("cpu"),
    weights_only=True
))
model.eval();

In [5]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

In [6]:
prompt = """Below is an instruction that describes a task. Write a response 
that appropriately completes the request.

### Instruction:
Convert the active sentence to passive: 'The chef cooks the meal every day.'
"""

In [7]:
from previous_chapters import (
    generate,
    text_to_token_ids,
    token_ids_to_text
)

def extract_response(response_text, input_text):
    return response_text[len(input_text):].replace("### Response:", "").strip()

torch.manual_seed(123)

token_ids = generate(
    model=model,
    idx=text_to_token_ids(prompt, tokenizer),
    max_new_tokens=35,
    context_size=BASE_CONFIG["context_length"],
    eos_id=50256
)

response = token_ids_to_text(token_ids, tokenizer)
response = extract_response(response, prompt)
print(response)

The meal is cooked every day by the chef.
